In [ ]:
import json
import numpy
import pandas
from pprint import pprint
pandas.options.display.max_colwidth = -1

In [ ]:
# runs in about 5 minutes 
with open('with_derived/events_with_derived_1930.json','r') as f:
    allgames = json.load(f)
print('ok')
print(len(allgames))
#pprint(allgames[2])
allgames_bk = allgames

In [ ]:
column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_team', 'sub_batting_order', 'sub_position']#, 'gameID']

show_columns_for_weirdness = ['event_in_game', 'inning', 'home_or_visitor', 'event', 'comment']

for i in range(0,len(allgames)):
    thisgamedict = allgames[i]
    thisgameevents_df = pandas.DataFrame(thisgamedict['events'], columns=column_list)
    if (numpy.mod(i,100) == 0):
        print('processing game {0:,.0f}...'.format(i))
    for j in range(0,len(thisgameevents_df)): #- 1):
        if (thisgameevents_df.iloc[j]['event_type'] != 'play'):
            thisgameevents_df.iloc[j]['inning'] = thisgameevents_df.iloc[j-1]['inning']
            thisgameevents_df.iloc[j]['home_or_visitor'] = thisgameevents_df.iloc[j-1]['home_or_visitor']
            
        if (thisgameevents_df.iloc[j-1]['event_type'] not in ('play','sub')):
            thisgameevents_df.iloc[j]['comment'] = thisgameevents_df.iloc[j-1]['comment'] + ' ' + thisgameevents_df.iloc[j]['comment']

    thisgameevents_df = thisgameevents_df[(thisgameevents_df['event_type'] == 'play') | (thisgameevents_df['event_type'] == 'sub')]
    thisgameevents_df = thisgameevents_df.drop('event_in_game',axis=1)
    thisgameevents_df = thisgameevents_df.reset_index(drop=True)
    thisgameevents_df.index.name = 'event_in_game'
    allgames[i]['events'] = thisgameevents_df.reset_index().values.tolist()

print('preparing to save file...')
#thisgameevents_df
with open('events_json/events_1930.json','w') as f:
    json.dump(allgames, f)
print('done')

In [ ]:
    # replace event types with number codes for fastness
#    thisgameevents_df['event_type'] = numpy.where((thisgameevents_df['event_type'] == 'play'),0,(thisgameevents_df['event_type']))
#    thisgameevents_df['event_type'] = numpy.where((thisgameevents_df['event_type'] == 'sub'),1,(thisgameevents_df['event_type']))
#    thisgameevents_df['event_type'] = numpy.where((thisgameevents_df['event_type'] == 'com'),2,(thisgameevents_df['event_type']))
#    thisgameevents_df['event_type'] = numpy.where((thisgameevents_df['event_type'] == 'badj'),3,(thisgameevents_df['event_type']))
#    thisgameevents_df['event_type'] = numpy.where((thisgameevents_df['event_type'] == 'padj'),4,(thisgameevents_df['event_type']))
#    thisgameevents_df['event_type'] = numpy.where((thisgameevents_df['event_type'] == 'badj'),5,(thisgameevents_df['event_type']))
#    thisgameevents_df['event_type'] = pandas.to_numeric(thisgameevents_df['event_type'])
    
